In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from transformers import AutoTokenizer

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:

df=pd.read_csv("./NewsSumm_perfect_clean.csv")

In [ ]:
df.head()

,newspaper_name,published_date,headline,article_text,human_summary,news_category,article_clean,summary_clean,article_tokens,summary_tokens,compression_ratio,cluster_id
0,Indian Express,2023-10-08,When Congress fell short of a majority despite...,As India-Canada relations are strained in the ...,"In summary, ""I am not thinking about the India...",International News,As India-Canada relations are strained in the ...,"In summary, ""I am not thinking about the India...",308,95,0.308442,75523
1,Indian Express,2023-10-16,"""I am not thinking about the India-Canada rift...",Isro chairman S Somanath on Sunday said expert...,"In summary, ""I am not thinking about the India...",Science and Technology,Isro chairman S Somanath on Sunday said expert...,"In summary, ""I am not thinking about the India...",434,95,0.218894,75523
2,Financial Express,2018-09-06,Andhra Pradesh to punish tax officials for unr...,In an attempt to minimize the harassment faced...,Andhra Pradesh has formed four nodal committee...,Business and Finance,In an attempt to minimize the harassment faced...,Andhra Pradesh has formed four nodal committee...,297,86,0.289562,20930
3,Indian Express,2011-04-03,The World At Our Feet,The headline of the Times of India on April ...,India defeated Sri Lanka in the final of the C...,Sports,"The headline of the Times of India on April 2,...",India defeated Sri Lanka in the final of the C...,175,63,0.360000,79366
4,Hindustan Times,2001-01-01,2001 Parliament attack: ‘A shot missed me and ...,"On December 13, 2001, a group of five armed m...",The 2001 Parliament attack was a major terrori...,terrorist attack,"On December 13, 2001, a group of five armed me...",The 2001 Parliament attack was a major terrori...,319,118,0.369906,143292


In [ ]:
cluster_sizes = df.groupby('cluster_id').size()

valid_clusters = cluster_sizes[cluster_sizes >= 2].index

df_multi = df[df['cluster_id'].isin(valid_clusters)].reset_index(drop=True)

print("Filtered rows:", len(df_multi))
print("Filtered clusters:", df_multi['cluster_id'].nunique())
print("Avg docs per cluster:",
      df_multi.groupby('cluster_id').size().mean())


Filtered rows: 4335
Filtered clusters: 2060
Avg docs per cluster: 2.104368932038835


In [ ]:
from sklearn.model_selection import train_test_split

clusters = df_multi['cluster_id'].unique()

train_clusters, temp_clusters = train_test_split(
    clusters, test_size=0.2, random_state=42
)

val_clusters, test_clusters = train_test_split(
    temp_clusters, test_size=0.5, random_state=42
)

train_df = df_multi[df_multi['cluster_id'].isin(train_clusters)]
val_df   = df_multi[df_multi['cluster_id'].isin(val_clusters)]
test_df  = df_multi[df_multi['cluster_id'].isin(test_clusters)]

print("Train clusters:", train_df['cluster_id'].nunique())
print("Val clusters:", val_df['cluster_id'].nunique())
print("Test clusters:", test_df['cluster_id'].nunique())


Train clusters: 1648
Val clusters: 206
Test clusters: 206


In [ ]:
def build_cluster_samples(df):
    samples = []
    for cid, group in df.groupby("cluster_id"):
        docs = []

        # sort only if column exists
        if "published_date" in group.columns:
            try:
                group = group.sort_values("published_date")
            except Exception:
                pass

        for _, row in group.iterrows():
            docs.append(f"[DOC]\n{row['article_clean']}")

        samples.append({
            "cluster_id": cid,
            "source": "\n".join(docs),
            "summary": group.iloc[0]['summary_clean']
        })
    return samples


In [ ]:
train_samples = build_cluster_samples(train_df)
val_samples   = build_cluster_samples(val_df)
test_samples  = build_cluster_samples(test_df)

print("Train samples:", len(train_samples))
print("Val samples:", len(val_samples))
print("Test samples:", len(test_samples))


Train samples: 1648
Val samples: 206
Test samples: 206


In [ ]:
!pip install -q transformers datasets accelerate sentencepiece


In [ ]:
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)

# =========================
# CONFIG
# =========================
MODEL_NAME = "google/long-t5-tglobal-base"

MAX_INPUT_LEN = 4096
MAX_TARGET_LEN = 256

RAW_SOURCE_CHAR_LIMIT = 12000
RAW_SUMMARY_CHAR_LIMIT = 2000

LR = 1e-5
EPOCHS = 6
BATCH_SIZE = 1
GRAD_ACC = 4
LOGGING_STEPS = 50

# =========================
# DATASET FORMAT
# train_samples = [{"source": "...", "summary": "..."}]
# val_samples   = [{"source": "...", "summary": "..."}]
# =========================

def clean_samples(samples):
    cleaned = []
    for s in samples:
        if len(s["source"].strip()) < 200:
            continue
        if len(s["summary"].strip()) < 30:
            continue
        cleaned.append(s)
    return cleaned

train_samples = clean_samples(train_samples)
val_samples   = clean_samples(val_samples)

train_ds = Dataset.from_list(train_samples)
val_ds   = Dataset.from_list(val_samples)

# =========================
# TOKENIZER
# =========================
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=False
)

# =========================
# MODEL
# =========================
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

model.gradient_checkpointing_enable()
model.config.use_cache = False

# =========================
# PREPROCESS
# =========================
def preprocess(batch):
    sources = [s[:RAW_SOURCE_CHAR_LIMIT] for s in batch["source"]]
    summaries = [s[:RAW_SUMMARY_CHAR_LIMIT] for s in batch["summary"]]

    model_inputs = tokenizer(
        sources,
        max_length=MAX_INPUT_LEN,
        truncation=True,
        padding=False
    )

    labels = tokenizer(
        text_target=summaries,
        max_length=MAX_TARGET_LEN,
        truncation=True,
        padding="max_length"
    )

    labels["input_ids"] = [
        [(t if t != tokenizer.pad_token_id else -100) for t in seq]
        for seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_data = train_ds.map(
    preprocess,
    batched=True,
    remove_columns=train_ds.column_names
)

val_data = val_ds.map(
    preprocess,
    batched=True,
    remove_columns=val_ds.column_names
)

# =========================
# DATA COLLATOR
# =========================
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# =========================
# TRAINING ARGS
# =========================
training_args = TrainingArguments(
    output_dir="./longt5_finetuned",
    overwrite_output_dir=True,

    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=GRAD_ACC,
    num_train_epochs=EPOCHS,

    learning_rate=LR,
    warmup_ratio=0.05,
    lr_scheduler_type="linear",

    max_grad_norm=1.0,
    label_smoothing_factor=0.0,

    fp16=False,
    bf16=False,

    logging_steps=LOGGING_STEPS,
    save_strategy="no",
    report_to="none"
)

# =========================
# TRAINER
# =========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# =========================
# SANITY CHECK (GPU SAFE)
# =========================
device = model.device
batch = next(iter(train_data))

out = model(
    input_ids=torch.tensor(batch["input_ids"]).unsqueeze(0).to(device),
    attention_mask=torch.tensor(batch["attention_mask"]).unsqueeze(0).to(device),
    labels=torch.tensor(batch["labels"]).unsqueeze(0).to(device)
)

print("Sanity loss:", out.loss.item())

# =========================
# TRAIN
# =========================
print("🚀 Training Long-T5 on Colab T4...")
trainer.train()

# =========================
# SAVE
# =========================
trainer.save_model("./longt5_finetuned")
tokenizer.save_pretrained("./longt5_finetuned")

print("✅ Training finished successfully!")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

Map:   0%|          | 0/206 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:896: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Sanity loss: 0.31884464621543884
🚀 Training Long-T5 on Colab T4...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:896: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss
50,2.300700
100,2.378200
150,2.297700
200,1.922200
250,1.835600
300,1.766900
350,1.837100
400,1.725700
450,1.697100
500,1.639400


✅ Training finished successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SAVE_DIR = "/content/drive/MyDrive/longt5_finetuned"

trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

('/content/drive/MyDrive/longt5_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/longt5_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/longt5_finetuned/spiece.model',
 '/content/drive/MyDrive/longt5_finetuned/added_tokens.json')